# Tutorial 04 - Homografías

Agenda:

- Transformaciones:
    - Afinidades
    - Homografías
- DLT
- RANSAC
- ejercicios

# Setup

Este tutorial se puede ejecutar local en Jupyter lab o utilizar Google Colab.

## En Google Colab
Este tutorial se provee junto con archivos de recursos dentro de un archivo ".zip".
En caso de ejecutar en Google Colab hay que:

1. Descomprimir el zip en algún lado
2. Subir el contenido del zip a Google Drive en alguna carpeta (por ejemplo `udesa/I308/tutoriales/tutorial_X`)
3. Abrir este notebook .ipynb

In [5]:
import os
import sys

# TODO: establecer el path en caso de trabajar con Colab
DRIVE_DIR = "TERCERO 2DO CUATRI/Tutorial Vision/tutorial_04"

# detecta si estamos corriendo en Google Colab
try:
  from google.colab import drive
  COLAB = True
except:
  COLAB = False

if COLAB:
    # monta Google Drive
    drive.mount('/content/drive')

    base_path = "/content/drive/MyDrive/"
    path = os.path.join(base_path, DRIVE_DIR)

    %cd {path}
    sys.path.append(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/TERCERO 2DO CUATRI/Tutorial Vision/tutorial_04


In [6]:
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'imp'

In [ ]:
# instalamos el paquete de utilidades
!pip install -qq git+https://github.com/udesa-vision/i308-utils.git

from i308_utils import imshow, show_images

# Coordenadas homogeneas

Las coordenadas homogéneas son una **extensión de las coordenadas cartesianas** utilizado en geometría proyectiva,
que introduce una coordenada adicional, normalmente denotada como $\widetilde{w}$, lo que nos permite expresar transformaciones lineales de coordenadas de forma unificada como una matriz.

Si tenemos una coordenada cartesiana $p$, su equivalente homogeneo es $\widetilde{p}$:


\begin{align}
  p &= \begin{bmatrix}
          x \\
          y \\
        \end{bmatrix}
    ≡ \begin{bmatrix}
          x \\
          y \\
          1 \\
        \end{bmatrix}
    ≡ \begin{bmatrix}
          \widetilde{w} . x \\
          \widetilde{w} . y \\
          \widetilde{w} \\
        \end{bmatrix}
    = \begin{bmatrix}
          \widetilde{x} \\
          \widetilde{y} \\
          \widetilde{w} \\
        \end{bmatrix}
    =  \widetilde{p}
\end{align}

<img src="https://raw.githubusercontent.com/udesa-vision/i308-resources/23e36da06db54dd731f3379268811b6ab318ac3a/tutoriales/tutorial_04/homo.svg"/>

# Dibujando coordenadas

Si armamos una lista de coordenadas queda determinada una figura.
Dibujémosla:

In [ ]:
import numpy as np
from homo_utils import plot_shape, homo, cart, apply_transform
from matplotlib import pyplot as plt

ori = np.array([
  (0, 0),
  (3, 0),
  (3, 2),
  (0, 2)
])

plt.figure(figsize=(6, 6))
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plot_shape(ori)
plt.grid()
plt.xlim(-1, 5)
plt.ylim(-1, 5)


plt.show()

In [ ]:
# en homo_utils.py se provee una funcion para convertir coordenadas
# cartesianas en homogeneas:
# ojo, estas coordenadas están definidas con filas.

homo(ori)

In [ ]:
p = np.array([
    [5., 2., 2.]
])

In [ ]:
cart(p)

In [ ]:
# y luego dadas coordenadas homogeneas la funcion que vuelve a coordenadas cartesianas.

cart(homo(ori))

# Transformaciones geométricas lineales (Warping)

Estas transformaciones mapean las posiciones de los píxeles de una imagen de origen en nuevas posiciones en las coordenadas de la imagen de destino, deformando la imagen.

Las estas transformaciones se pueden implementar usando una matriz T:

$ p_2 = T . p_1 $

## Afinidades

Las tranformaciones afines en general tienen la forma:

$
A = \begin{bmatrix}
    a_{11} & a_{12} & a_{13} \\
    a_{21} & a_{22} & a_{23} \\
    0 & 0 & 1 \\
\end{bmatrix}
$

¿qué efecto tiene variar cada $a_{ij}$?



### Escalado

Podemos estirar o comprimir coordenadas en cada eje usando un factor de escala.

$ p2 = S . p1 $

$
S = \begin{bmatrix}
    s_x & 0 & 0 \\
    0 & s_y & 0 \\
    0 & 0 & 1 \\
\end{bmatrix}
$

In [ ]:
# TODO:
# - aplicar una transformación S para escalar las coordenadas dadas

sx = 2
sy = 0.5
S = np.array([
    [sx, 0, 0],
    [0, sy, 0],
    [0, 0, 1]]
)

dst = apply_transform(ori, S)

plt.figure(figsize=(7, 7))
plot_shape(ori, w=3, fill='blue')
plot_shape(dst, w=3, fill='orange')
# plt.axis('equal')

plt.grid()
plt.xlim(-1, 7)
plt.ylim(-2, 6)

plt.show()



### Rotación

Podemos rotar las coordenadas dado el ángulo $θ$.

$ p_2 = R . p_1 $

$
R = \begin{bmatrix}
    cos(θ) & -sin(θ) & 0 \\
    sin(θ) & cos(θ) & 0 \\
    0 & 0 & 1 \\
\end{bmatrix}
$

In [ ]:
# TODO:
# - aplicar una transformación de rotación R para rotar las coordenadas
#    45 grados en sentido anti-horario.
# - graficar los resultados.
# - cómo será R^(-1) (la rotación inversa)?


angle = 45
theta = np.pi * angle / 180.0

R = np.array([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
])

dst = apply_transform(ori, R)

plt.figure(figsize=(7, 7))
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plot_shape(ori, w=3, fill='blue')
plot_shape(dst, w=3, fill='orange')
plt.axis('equal')
plt.xlim(-3, 4)
plt.grid()
plt.show()


### Skew / Shear (en español??)

Skew deforma rectángulos en paralelogramos.

$ p_2 = M . p_1 $

$
M = \begin{bmatrix}
    1 & m_x & 0 \\
    m_y & 1 & 0 \\
    0 & 0 & 1 \\
\end{bmatrix}
$

In [ ]:
# TODO:
# - aplicar skew para deformar la figura original
# - graficar los resultados.

mx = 0.0
my = 0.3

M = np.array([
    [1, mx, 0],
    [my, 1, 0],
    [0, 0, 1]
])

dst = apply_transform(ori, M)

plt.figure(figsize=(7, 7))
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plot_shape(ori, w=3, fill='blue')
plot_shape(dst, w=3, fill='orange')
# plt.axis('equal')
plt.xlim(-1, 5)
plt.ylim(-1, 5)
plt.grid()
plt.show()

### Traslación

Una translación permite desplazar las coordenadas en x y en y.

$ p_2 = T . p_1 $

$
T = \begin{bmatrix}
    1 & 0 & t_x \\
    0 & 1 & t_y \\
    0 & 0 & 1 \\
\end{bmatrix}
$

In [ ]:
tx = 2
ty = 3

T = np.array([
    [1, 0, tx],
    [0, 1, ty],
    [0, 0, 1]
])

dst = apply_transform(ori, T)

plt.figure(figsize=(7, 7))
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plot_shape(ori, w=3, fill='blue')
plot_shape(dst, w=3, fill='orange')
# plt.axis('equal')
plt.xlim(-1, 6)
plt.ylim(-1, 6)
plt.grid()
plt.show()

### Composicion de transformaciones

Si tengo dos transformaciones afines

$ p_2 = A_{12} . p_1 $

$ p_3 = A_{23} . p_2 $

puedo combinar ambas transformaciones en una nueva transformación simplemente multiplicando las matrices:

$ A_{13} = A_{23} . A_{12} $

OBS: $ A_{13} $ es también una *Afinidad*

In [ ]:
A = S
A = R
A = np.dot(R, S)
A = np.dot(S, R)

# x' = S x
# x'' = R x'
#
# x'' = R (S x) = (R S) x = A x
# => x'' = A x


dst = apply_transform(ori, A)

plt.figure(figsize=(7, 7))
plot_shape(ori, w=3, fill='blue')
plot_shape(dst, w=3, fill='orange')
# plt.axis('equal')
plt.xlim(-3, 7)
plt.ylim(-3, 7)
plt.grid()
plt.show()

### Afinidad inversa

dada una afinidad A, podemos invertir A, o bien:

separamos la parte lineal, de la traslación:

$
A = \begin{bmatrix}
    a_{11} & a_{12} & a_{13} \\
    a_{21} & a_{22} & a_{23} \\
    0 & 0 & 1 \\
\end{bmatrix}
$
$ \rightarrow $
$
M = \begin{bmatrix}
    a_{11} & a_{12} \\
    a_{21} & a_{22} \\
\end{bmatrix}
$,
$
t = \begin{bmatrix}
    a_{13}  \\
    a_{23}  \\
\end{bmatrix}
$

Luego

$
A^{-1} = \begin{bmatrix}
    M^{-1} & -M^{-1} . t \\
    0 & 1 \\
\end{bmatrix}
$

In [ ]:
# dada una afinidad, A
# no es dificil encontrar la inversa de una afinidad.


from homo_utils import affine_inv

A = np.array([
    [1, 2, 1],
    [1, 1, 2],
    [0, 0, 1]]
)

# vamos del original con a dst con la afinidad A,
dst = apply_transform(ori, A)

# obtenemos la afinidad inversa
A_inv = affine_inv(A)

# aplicamos sobre dst la afinidad inversa
ori_inv = apply_transform(dst, A_inv)

# - ori_inv es igual a ori,
# - A^(-1) . A = I,
print(np.dot(A_inv, A))

plt.figure(figsize=(7, 7))
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plot_shape(ori_inv, w=3, fill='blue')
plot_shape(dst, w=3, fill='orange')
# plt.axis('equal')
plt.xlim(-1, 10)
plt.ylim(-1, 10)
plt.grid()
plt.show()

### Resumiendo
Las tranformaciones afines en general tienen la forma:

$
A = \begin{bmatrix}
    a_{11} & a_{12} & a_{13} \\
    a_{21} & a_{22} & a_{23} \\
    0 & 0 & 1 \\
\end{bmatrix}
$

- las lineas paralelas permanecen paralelas
- la composicion de afinidades me da afinidades

¿Qué pasa si $A_{31}$ ó $A_{32}$ son distintos de cero?

## Transformaciones proyectivas (Homografías)

Las homografías son transformaciones más generales que permiten incluir distorsiones de perspectiva.

Nos permiten transformar puntos de una imagen a otra como ocurre cuando las imágenes se toman desde diferentes puntos de vista o ángulos.

$ \widetilde{p_2} = H . \widetilde{p_1} $

$
H = \begin{bmatrix}
    h_{11} & h_{12} & h_{13} \\
    h_{21} & h_{22} & h_{23} \\
    h_{31} & h_{32} & h_{33} \\
\end{bmatrix}
$

In [ ]:
# TODO:

H = np.array([
    [  2,   0, 2],
    [  0,   2, 1],
    [0.1, 0.1, 1]
])

dst = apply_transform(ori, H)

plt.figure(figsize=(7, 7))
plot_shape(ori, w=3, fill='blue')
plot_shape(dst, w=3, fill='orange')
#plt.axis('equal')
plt.xlim(-1, 7)
plt.ylim(-1, 7)
plt.grid()
plt.show()

In [ ]:
ori

In [ ]:
dst.round(1)

**Encontrando la Homografía**

Si yo tengo pares de correspondencias, $ori[i]$ <-> $dst[i]$

En este ejemplo: $(0, 0)$ <-> $(2, 1)$, $(0, 2)$ <-> $(1.7, 4.2)$, ...



- ¿Cómo puedo encontrar la H que los relaciona?
- ¿Esta H es única?
- ¿Cuántos pares de correspondencias necesito?


# Direct Linear Transformation - DLT.

las homografías establecen equivalencias entre puntos del espacio proyectivo.

Estas equivalencias valen para cualquier escalar $k \neq 0$ por el que multipliquemos.

$ H . x_1 ≡ k . H . x_2 $

Con lo cual si nosotros normalizamos H, dividendo $h_{ij}$ por $h_{33}$ obtenemos:

$
H = \begin{bmatrix}
    h'_{11} & h'_{12} & h'_{13} \\
    h'_{21} & h'_{22} & h'_{23} \\
    h'_{31} & h'_{32} & 1 \\
\end{bmatrix}
$

Esto quiere decir que las homografías tienen DoF=8.

Si identificamos coordenadas de un objeto (plano) un un marco de referencia (ejemplo en la imagen 1) y las coordenadas correspondientes otra imagen.

¿Cómo podríamos estimar H que transforme puntos de una imagen en la otra?

Tenemos 8 incógnitas, cuántos pares de correspondencias necesitamos?



$X$, $X'$, son conocidos, $H$ es desconocida.

$
X' = H . X
$

$
\begin{bmatrix}
    x' \\
    y' \\
    1 \\
\end{bmatrix}
= H .
\begin{bmatrix}
    x \\
    y \\
    1 \\
\end{bmatrix}
= \begin{bmatrix}
    h_{11} x + h_{12} y + h_{13} \\
    h_{21} x + h_{22} y + h_{23} \\
    h_{31} x + h_{32} y + 1 \\
\end{bmatrix}
$

luego:

$ x' = \frac{h_{11} x + h_{12} y + h_{13}}{h_{31} x + h_{32} y + 1} $

$ y' = \frac{h_{21} x + h_{22} y + h_{23}}{h_{31} x + h_{32} y + 1} $

con un poco de manejo algebraico:

$
\begin{cases}
    -h_{11} x - h_{12} y - h_{13} + x' . x . h_{31} + y . x' . h_{32} = x' \\
    -h_{21} x - h_{22} y - h_{23} + y' . x . h_{31} + y . y' . h_{32} = y'
\end{cases}
$

sistema que podemos expresar matricialmente como:

$
\begin{bmatrix}
    -x_{(i)} & -y_{(i)} & -1 & 0 & 0 & 0 & x_{(i)} . x_{(i)}' & y_{(i)} . x_{(i)}' \\
    0 & 0 & 0 & -x & -y & -1 & x . y' & y . y' \\
\end{bmatrix}
.
\begin{bmatrix}
     h_{11}' \\
     h_{21}' \\
     h_{31}' \\
     h_{21}' \\
     h_{22}' \\
     h_{23}' \\
     h_{31}' \\
     h_{32}'
\end{bmatrix}
=
\begin{bmatrix}
    x_{(i)}' \\
    y_{(i)}'
\end{bmatrix}
$


$ A . h = b $

In [ ]:
def dlt(ori, dst):

    # Construct matrix A and vector b
    A = []
    b = []
    for i in range(4):
        x, y = ori[i]
        x_prima, y_prima = dst[i]
        A.append([-x, -y, -1, 0, 0, 0, x * x_prima, y * x_prima])
        A.append([0, 0, 0, -x, -y, -1, x * y_prima, y * y_prima])
        b.append(x_prima)
        b.append(y_prima)

    A = np.array(A)
    b = np.array(b)

    # resolvemos el sistema de ecuaciones A * h = b
    # el sistema es de 8x8, por lo que podemos resolverlo si A es inversible

    # resuelve el sistema de ecuaciones para encontrar los parámetros de H
    H = -np.linalg.solve(A, b)

    # agrega el elemento h_33
    H = np.hstack([H, [1]])

    # reorganiza H para formar la matrix en 3x3 to form the 3x3 homography matrix
    H = H.reshape(3, 3)

    return H

In [ ]:
H2 = dlt(ori, dst).round(6)

In [ ]:
H

In [ ]:
H2

# DLT con OpenCV

Como hacemos "lo mismo" con OpenCV?

In [ ]:
import cv2

In [ ]:
H3, _ = cv2.findHomography(ori, dst)
H3.round(6)

In [ ]:
dst = apply_transform(ori, H2)

plt.figure(figsize=(7, 7))
plot_shape(ori, w=3, fill='blue')
plot_shape(dst, w=3, fill='orange')
# plt.axis('equal')
plt.xlim(-1, 7)
plt.ylim(-1, 7)
plt.grid()
plt.show()

# Warping en Imágenes

Hasta ahora las transformaciones las aplicamos en coordenadas.

¿Cómo podemos transformar imagenes?


In [ ]:
import cv2

from matplotlib import pyplot as plt

In [ ]:
img = cv2.imread("res/lenna.png", 0)

In [ ]:
img.shape

In [ ]:
imshow(img)

In [ ]:
# TODO:
# - usando producto de numpy, escalar
#    los pixeles de la imagen con una transformacion S: sx = 1.5, sy = 1.3
# - y graficar el resultado
# - con qué problemas nos encontramos?


In [ ]:
import numpy as np

sx = 2.0
sy = 1.1

S = np.array([[sx, 0, 0], [0, sy, 0], [0, 0, 1]])

# dimensiones de la imagen
h, w = img.shape
x = np.arange(w)
y = np.arange(h)

# armo todas las coordenadas de la imagen
xx, yy = np.meshgrid(x, y)
ones = np.ones((h, w))

# y las expreso en coordenadas homogeneas
homogeneus_indices = np.stack(
    (xx.reshape(-1),
     yy.reshape(-1),
     ones.reshape(-1)
))

# la transformación lleva las coordenadas a las coordenadas escaladas
transformed = np.dot(S, homogeneus_indices)

# obtengo cartesianas de las transformadas homogeneas
x_transformed = transformed[0, :].reshape((h, w))
y_transformed = transformed[1, :].reshape((h, w))

output_h = int(sy * h) # + 1
output_w = int(sx * w) # + 1
output_img = np.zeros((output_h, output_w), dtype='uint8')

y_transformed = np.round(y_transformed).astype(int)
x_transformed = np.round(x_transformed).astype(int)

output_img[
  y_transformed,
  x_transformed
] = img[yy, xx]


# ¿Qué está pasando?
plt.figure(figsize=(12, 4))
plt.imshow(output_img, cmap='gray', interpolation='none')
plt.axis('off')
plt.show()

In [ ]:
img[0:3, 0:3]
output_img[0:3, 0:6]
transformed[:, :3]
x_transformed[:3, :10]

In [ ]:
# Para no tener huecos, vamos al revés, usamos la transformacion inversa.
# es decir para cada pixel de la imagen de destino,
# vamos a buscar los valores de gris en las coordenadas mas cercanas inversas
# de la imagen original.

S_inv = np.array([
    [1/sx, 0, 0],
    [0, 1/sy, 0],
    [0, 0, 1]
])

In [ ]:
# S_inv es la inversa?
np.dot(S, S_inv)

In [ ]:
output_h = int(sy * h)
output_w = int(sx * w)

x = np.arange(output_w)
y = np.arange(output_h)

xx, yy = np.meshgrid(x, y)
ones = np.ones((output_h, output_w))

homogeneus_indices = np.stack([
    xx.reshape(-1),
    yy.reshape(-1),
    ones.reshape(-1)
])

orig = np.dot(S_inv, homogeneus_indices)

x_orig = orig[0, :].round().astype(int)
y_orig = orig[1, :].round().astype(int)

x_orig = np.clip(x_orig, 0, w - 1)
y_orig = np.clip(y_orig, 0, h - 1)

output_img2 = np.zeros((output_h, output_w), dtype='uint8')
output_img2[
    yy.reshape(-1),
    xx.reshape(-1)
] = img[y_orig, x_orig]

plt.figure(figsize=(12, 4))
plt.imshow(output_img2, cmap='gray', interpolation='none')
plt.axis('off')
plt.show()

# Warping con OpenCV

OpenCV nos provee las funciones [`cv2.warpAffine`](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#ga0203d9ee5fcd28d40dbc4a1ea4451983)

y [`cv2.warpPerspective`](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87)

que resuelve los temas de interpolación por nosotros.



In [ ]:
A = S[0:2]
A

In [ ]:
# TODO:
# usando cv2.warpAffine escalar la imagen
#


A = S[0:2]

new_h, new_w = int(sy * h), int(sx * w)

# Aplica la transformación afín usando opencv cv2.warpAffine
scaled_img = cv2.warpAffine(img, A, (new_w, new_h), flags=cv2.INTER_CUBIC)

show_images([
  output_img,
  scaled_img
], ["naive", "cv"])

In [ ]:
A

In [ ]:
H

In [ ]:
H = np.array([
    [  2,   0,   50],
    [  0,   2,   100],
    [  0.001,   0.002,   1.]
])

ori = np.array([
  (0, 0),
  (440, 0),
  (440, 440),
  (0, 440)
])

dst = apply_transform(ori, H)

plt.figure(figsize=(7, 7))
plot_shape(ori, w=3, fill='blue')
plot_shape(dst, w=3, fill='orange')
plt.xlim(-100, 700)
plt.ylim(-100, 700)

# plt.axis('equal')

plt.grid()
plt.show()


In [ ]:
ori

In [ ]:
img.shape

In [ ]:
# TODO:
# usando cv2.warpPerspective, transformar la imagen.

# 1. cómo calculamos dsize?
# 2. cómo podemos ajustar la posición transformada resultante para no "desperdiciar" píxeles?

dsize = (880, 880) # ?
imshow(cv2.warpPerspective(img, H, dsize=dsize))

In [ ]:
h, w = img.shape

# defino los bordes de la imagen
corners = np.array([
    [0, 0],
    [w, 0],
    [w, h],
    [0, h]
], dtype=np.float32)


corners_homo = np.hstack([
    corners,
    np.ones((4, 1))]
)


transformed = np.dot(H, corners_homo.T)
transformed /= transformed[2, :]
transformed

transformed = transformed[:2, :].T

# encontramos coordenadas minimas y máximas de los corners transformados.
min_x, min_y = np.min(transformed, axis=0).round().astype(int)
max_x, max_y = np.max(transformed, axis=0).round().astype(int)

# el tamaño final "óptimo" lo puedo calcular
output_width = int(np.ceil(max_x - min_x))
output_height = int(np.ceil(max_y - min_y))


# ajusta la parte de traslación de la matrix de homografía
translation_matrix = np.array([
    [1, 0, -min_x],
    [0, 1, -min_y],
    [0, 0, 1]
])

# componemos H con la traslación
adjusted_H = np.dot(translation_matrix, H)

# warpeamos la imagen
output_img = cv2.warpPerspective(img, adjusted_H, (output_width, output_height))
imshow(output_img)

In [ ]:
output_img.shape

# Ejercicio 1 - DLT

Futbol futbol futbol!

¿El jugador está en posición adelantada?

<img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_04/futbol.png?raw=true" />

Sería más fácil determinarlo si pudiéramos "warpear" la cancha para obtener una imagen como vista desder arriba (aka bird's-eye view)

¿Cómo podríamos lograr eso?

In [ ]:
def draw_shape(img, shape):

    draw = img.copy()
    is_closed = True
    color = (255, 0, 0)
    thickness = 5

    draw = cv2.polylines(draw, shape, is_closed, color, thickness)
    return draw

In [ ]:
# Cómo transformo putos con la homografía?
# esta funcion recibe la H, y puntos,
# pasa los puntos a homogéneo, los transforma, y luego vuelve al espacio in-homogeneo
def transform(H, shape):

    # convierto el BB al homogeneo
    num_points = shape.shape[1]
    points_homo = np.hstack((shape.reshape(-1, 2), np.ones((num_points, 1))))

    # Apply homography matrix to points
    transformed_points_homo = np.dot(H, points_homo.T).T

    # Convert back to Cartesian coordinates by normalizing with the last coordinate
    transformed_points = (transformed_points_homo[:, :2].T / transformed_points_homo[:, 2]).T

    pts = transformed_points.round().reshape(1, -1, 2).astype('int32')
    return pts


In [ ]:
# Busco 4 puntos en el piso que van a servir como pares de correspondencia
corners = [
    (1265, 994),
    (1604, 964),
    (2472, 1182),
    (2083, 1222),
]
corners = np.array(corners).reshape(1, -1, 2).astype('int32')

# Posiciones de los jugadores
players = [
    (477, 1443),
    (694, 1008),
    (1552, 1437),

    (696, 1210), # referee

    (1906, 1047), # GK

    (1310, 1320),
    (1978, 1152),
    (1692, 1150)

]
players_color = [
    'b',
    'b',
    'b',
    'r',
    'y',
    'c',
    'c',
    'c',

]
players = np.array(players)

# Grafiquemos la idea:
img = cv2.imread("res/futbol.png")
fig, ax = imshow(
    draw_shape(img, corners),
    figsize=(16, 8),
    show=False
)
area = corners.reshape(-1, 2)
ax.scatter(players[:, 0], players[:, 1], color=players_color, s=128, marker='x')
ax.scatter(area[:, 0], area[:, 1], color=['r', 'g', 'b', 'm'], s=64)

plt.show()

In [ ]:
# TODO:
# Warpear el campo de juego para que se vea desde arriba
# y sea más fácil por ejemplo detectar una posición adelantada.

# 1. Usando DLT
# 2. Usando OpenCV findHomography

# Creemos la imagen resultante.
rgb = (30, 180, 50)
ret = np.zeros((1800, 2048, 3), dtype='uint8')
w, h = ret.shape[1], ret.shape[0]

# genero puntos de correspondencia en la imagen resultante
corners_rect = np.array([[
    [w-200, 900],
    [w-1, 900],
    [w-1, 900+450],
    [w-200, 900+450]
]], dtype=np.int32)


fig, ax = imshow(
    draw_shape(ret, corners_rect),
    figsize=(10, 8),
    show=False
)
area = corners_rect.reshape(-1, 2)
ax.scatter(area[:, 0], area[:, 1], color=['r', 'g', 'b', 'm'], s=64)
plt.show()



In [ ]:

# 1. Usando DLT, encontramos la H
H = dlt(
    corners.reshape(-1, 2),
    corners_rect.reshape(-1, 2)
)

# 2. Usando OpenCV findHomography
# H = cv2.findHomography(corners, corners_rect)[0]

transformed_players = transform(H, players.reshape(2, -1))
transformed_players = transformed_players.reshape(-1, 2)

ret = cv2.warpPerspective(
    img, H,
    (w, h)
)

fig, ax = imshow(
    draw_shape(ret, corners_rect),
    figsize=(10, 8),
    show=False
)
ax.scatter(
    transformed_players[:, 0],
    transformed_players[:, 1],
    color=players_color,
    marker='+',
    s=96
)
ax.scatter(area[:, 0], area[:, 1], color=['r', 'g', 'b', 'm'], s=64)
plt.show()

In [ ]:
# OBS:
# - los puntos correspondientes no tienen por que vivir en un rectángulo.
# - si yo tengo un "template" del mundo, entonces puedo pasar puntos de la imagen a unidades métricas!
# - usando ese template podria volver para atrás y encontrar los pixeles donde caerían en la imagen.



In [ ]:
template = cv2.imread("res/soccer_template.png")

imshow(template)

In [ ]:
crop = template[:, 900:, :]
crop = cv2.rotate(crop, cv2.ROTATE_90_CLOCKWISE)
imshow(crop)
cv2.imwrite("res/soccer_template_crop.jpg", crop)

In [ ]:
corners = np.array([[
    (803, 762),
    #(557, 1082),
    (154, 942),
    (2472, 1182),
    (1820, 1538),
]], dtype='int32')


# Grafiquemos la idea:
fig, ax = imshow(
    img, figsize=(16, 8), show=False
)
keypoints = corners.reshape(-1, 2)
ax.scatter(players[:, 0], players[:, 1], color=players_color, s=128, marker='x')
ax.scatter(keypoints[:, 0], keypoints[:, 1], color=['r', 'g', 'b', 'm'], s=64)

plt.show()

In [ ]:
target_points = np.array([[
    (749, 350),
    (604, 148),
    (308, 350),
    (194, 148),
]], dtype='int32')

tp = target_points.reshape(-1, 2)

fig, ax = imshow(
    crop,
    figsize=(10, 8),
    show=False
)
ax.scatter(tp[:, 0], tp[:, 1], color=['r', 'g', 'b', 'm'])
plt.show()

target_w, target_h = crop.shape[1], crop.shape[0]
# 2. Usando OpenCV findHomography
H = cv2.findHomography(corners, target_points)[0]

ret = cv2.warpPerspective(
    img, H,
    (target_w, target_h)
)

transformed_players = transform(H, players.reshape(2, -1))
transformed_players = transformed_players.reshape(-1, 2)


fig, ax = imshow(
    ret,
    figsize=(10, 8),
    show=False
)
ax.scatter(
    transformed_players[:, 0],
    transformed_players[:, 1],
    color=players_color,
    marker='+',
    s=32
)
plt.show()


In [ ]:
# Como podríamos encontrar el punto del penal?
# obs el template que usamos no está a escala, pero...
# https://qph.cf2.quoracdn.net/main-qimg-583149598cc6c7737827a100ea456103-lq

template_points = np.array([[
    (399, 240),

]], dtype='int32')

tp = template_points.reshape(-1, 2)

fig, ax = imshow(
    crop, figsize=(10, 8), show=False
)
ax.scatter(tp[:, 0], tp[:, 1], color='r')
plt.show()

# Buscamos la H inversa!
H_inv = np.linalg.inv(H)

reprojected = transform(H_inv, template_points)

fig, ax = imshow(
    img, figsize=(16, 8), show=False
)
rep_pts = reprojected.reshape(-1, 2)
ax.scatter(rep_pts[:, 0], rep_pts[:, 1], color='r', s=32)

plt.show()


# Ejemplo OCR de tarjetas de crédito

In [ ]:
# Otro ejemplo "OCR":

card = cv2.imread("res/credit_card.jpg")
points = np.array([
    (50, 225),
    (830, 29),
    (968, 514),
    (185, 720),
])

card = cv2.imread("res/credit_card2.jpeg")
points = np.array([
    (41, 22),
    (573, 45),
    (566, 398),
    (47, 476),
])

fig, ax = imshow(card, show=False)
ax.scatter(points[:, 0], points[:, 1], color=['r', 'g', 'b', 'c'], marker='+')
plt.show()

In [ ]:
target = np.zeros((380, 600, 3), dtype='uint8')
w, h = target.shape[1], target.shape[0]

target_corners = np.array([
    [0, 0], [w, 0], [w, h], [0, h]
], dtype='float32')

points = points.astype('float32')

H = cv2.findHomography(points, target_corners)[0]

result = cv2.warpPerspective(card, H, (w, h))

imshow(result)

Ideas:
- tengo el layout de la tarjeta que es conocido. Entonces puedo buscar posiciones específicas en la imagen warpeada.
- OCR-A caracteres especiales, puedo usar template matching


In [ ]:
chars = cv2.imread("res/ocr-a.jpeg", cv2.IMREAD_GRAYSCALE)
chars = 255 - chars

gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

imshow(chars)
plt.show()
imshow(gray)
plt.show()



In [ ]:
cc = cv2.connectedComponentsWithStats(chars)
num_labels, labels, stats, centroids = cc

show_chars = cv2.cvtColor(chars, cv2.COLOR_GRAY2BGR)
img_chars = []
text_chars = []
cur_char = 0
for i in range(1, num_labels):  # start from 1 to skip the background
    # Extract the stats: x, y, width, height, and area
    x, y, w, h, area = stats[i]
    # print(x, y, w, h)
    # Draw bounding box around each character
    if area < 10 * 10:
        continue
    show_chars = cv2.rectangle(show_chars, (x, y), (x + w, y + h), (0, 255, 0), 5)
    crop_char = chars[y:y+h, x:x+w]
    img_chars.append(crop_char)
    text_chars.append(str(cur_char))
    cur_char += 1
imshow(show_chars)

show_images(img_chars, text_chars)

In [ ]:
_, th = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

cc2 = cv2.connectedComponentsWithStats(th)
num_labels, labels, stats, centroids = cc2

found_chars = result.copy()

def resize(img, w):
    w_, h_ = img.shape[1], img.shape[0]
    h = int(h_ * w / w_)
    return cv2.resize(img, (w, h))

font_scale = 1.1
font = cv2.FONT_HERSHEY_SIMPLEX
font_color = (0, 255, 0)
font_thickness = 2

cur_char = 0
for i in range(1, num_labels):  # start from 1 to skip the background
    # Extract the stats: x, y, width, height, and area
    x, y, w, h, area = stats[i]
    # print(x, y, w, h)
    # Draw bounding box around each character
    if (area < 12 * 12) or (area > 30 * 30):
        continue
    crop_cc = th[y:y+h, x:x+w]
    distances = []

    for char_img, char_txt in zip(img_chars, text_chars):
        resized = cv2.resize(crop_cc, (char_img.shape[1], char_img.shape[0]))
        diff = np.linalg.norm((resized.astype(float) - char_img.astype(float)))
        distances.append(diff)
    found_index = np.argmin(distances)
    if distances[found_index] < 15000:

        char_txt = text_chars[found_index]
        found_chars = cv2.rectangle(found_chars, (x, y), (x + w, y + h), (0, 255, 0), 1)

        found_chars = cv2.putText(
            found_chars,
            char_txt, (x, y),
            font,
            font_scale,
            font_color,
            font_thickness,
            cv2.LINE_AA
        )

imshow(found_chars)


# Homografía a partir de Matches de features

Si partimos de dos imágenes, calculamos descriptores invariantes locales, hacemos matching (lo que vimos la clase pasada)...

- ¿cómo podríamos encontrar una homografía que relacione ambas imágenes?
- ¿qué precondiciones necesito para que eso funcione?



In [ ]:
img
img1 = cv2.imread("res/img1.jpg")
# img2 = cv2.imread("res/img2.ppm")
img2 = cv2.imread("res/img4.jpg")


In [ ]:
show_images([img1, img2])

In [ ]:
# creamos el detector
algo = cv2.SIFT_create()

# detectamos keypoints y computamos los descriptores
kp1, des1 = algo.detectAndCompute(img1, None)
kp2, des2 = algo.detectAndCompute(img2, None)

In [ ]:
# matcheamos los descriptores
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
matches = bf.match(des1, des2)


In [ ]:
# dibujamos los matches.

def bw(img):
    # pasa a "blanco y negro" la imagen
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

red = (0, 0, 255)
green = (0, 255, 0)
blue = (255, 0, 0)
yellow = (0, 255, 255)

draw_params = dict(
    matchColor = (255, 0, 0),
    singlePointColor = (0, 0, 255),  # draw unmatched keypoints in red color
)
res_img = cv2.drawMatches(
    bw(img1), kp1,
    bw(img2), kp2,
    matches,
    None,
    matchColor=blue,
    singlePointColor=red
)

imshow(res_img, figsize=(12, 6))


In [ ]:
matches_knn2 = bf.knnMatch(des1, des2, k=2)

good_matches = []
for m, n in matches_knn2:
    if m.distance < 0.5 * n.distance:
        good_matches.append(m)

good_matches = sorted(good_matches, key=lambda x: x.distance)
# good_matches = good_matches[:64]

res_img = cv2.drawMatches(
    bw(img1), kp1,
    bw(img2), kp2,
    good_matches,
    None,
    matchColor=yellow,
    singlePointColor=red
)

imshow(res_img, figsize=(12, 6))

# Estos son los good_matches:

A partir de estos matches...

¿cómo podemos armar una homografía que mapee los puntos de la imagen de la izquierda en los de la derecha?

- DLT me resuelve si tengo exactamente 4 pares, pero acá tengo N >> 4.
- Además tengo mucho ruido matches buenos (inliers) y muchos matches malos (outliers)



In [ ]:
# Y si tomo los 4 mejores?
use_matches = good_matches[:4]


In [ ]:
res_img = cv2.drawMatches(
    bw(img1), kp1,
    bw(img2), kp2,
    use_matches,
    None,
    matchColor=yellow,
    singlePointColor=red
)

imshow(res_img, figsize=(12, 6))

In [ ]:
def points_from_matches(kp1, kp2, matches):
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])
    return pts1, pts2

pts1, pts2 = points_from_matches(kp1, kp2, use_matches)

# computo H usando DLT
H = dlt(pts1, pts2)

¿Qué tan buena será esa H?


In [ ]:
# Ejemplo si me armo un bounding box.
# cómo se transforma via H?
shape = np.array([[
    (180, 120),
    (450, 120),
    (450, 470),
    (180, 470),
]], np.int32)

In [ ]:
imshow(
    draw_shape(img1, shape)
)

In [ ]:
transformed_shape = transform(H, shape)

show_images([
    draw_shape(img1, shape),
    draw_shape(img2, transformed_shape)
])

- En la imagen izquierda el "monigote" está enmarcado completamente.
Y en la imagen derecha?

- La H que encontramos es buena? ¿Cómo podemos medir la calidad de H?



## Error de reproyección

Ahora que tengo un modelo (H), puedo testearo contra los datos y medir un error.
¿Qué error?
El error va a estar dado por la consistencia geométrica...

Si los puntos matcheados son los mismos, y viven en un plano (la pared), entonces al transformarlos deberían caer en el mismo plano transformado. Además su disposición espacial en el plano debería ser la misma.

Miremos un match es de la forma $ (x, \hat{x}) $, donde $x$ es la posicion en píxeles en la imagen izquierda, y $ \hat{x} $ es la posición correspondiente en píxeles de la imagen derecha.

Si la H es buena, y el match es correcto, debería pasar que el punto $ x $ transformado por H caiga muy cerca de la posición correspondiente:

$ x' = H x $

$ | x' - \hat{x} | \approx 0 $

Una métrica posible podría ser el RMSE que combina todos los errores individuales:

$ e_i = \| x'_i - \hat{x_i} \|_2  $

$
E = \sqrt{ \frac{1}{N} \sum_{i=1}^{N} e_i^2 }
$

In [ ]:
# si la H encontrada es buena, me mapea puntos de la izquierda a la derecha correctamente
def compute_reprojection_distances(H, pts1, pts2):
    # convierte pts1 a coordenadas homogéneas
    pts1_hom = np.hstack([pts1, np.ones((len(pts1), 1))])

    # proyecta pts1 usando H
    pts1_proy = np.dot(H, pts1_hom.T).T
    pts1_proy = pts1_proy[:, :2] / pts1_proy[:, 2].reshape(-1, 1)  # normaliza

    # calcular la distancia l2 entre los puntos proyectados y pts2
    dist = np.linalg.norm(pts1_proy - pts2, axis=1)
    return dist

def reprojection_error(H, pts1, pts2):
    """Calcula el error de reproyección."""
    dist = compute_reprojection_distances(H, pts1, pts2)

    # calcula RMSE
    rmse = np.sqrt(np.mean(dist**2))

    return rmse

In [ ]:
# uso todos los good-matches para calcular el error de reproyeccion
pts1, pts2 = points_from_matches(kp1, kp2, good_matches)

reprojection_error(H, pts1, pts2)


## Matches Inliers vs Outliers

Dada la homografía $H$ y un threshold $th$, decimos que un match $(x, \hat{x})$ es:


- **inlier** si $ \| x' - \hat{x} \|_2 \leq th $
- **outlier** else

en donde:
$ x' = H . x $


In [ ]:
def compute_inliers(H, pts1, pts2, thresh=5.0):

    dist = compute_reprojection_distances(H, pts1, pts2)
    # genera una máscara con los inliers
    return (dist < thresh).astype(int)



In [ ]:

# me fijo de los good matches cuáles son inliers, usando la H anterior

inliers_mask = compute_inliers(H, pts1, pts2, thresh=5.0)

#  imagen inicial para dibujar los matches
matches_img = cv2.drawMatches(bw(img1), [], bw(img2), [], [], None)

# dibujo inliers
cv2.drawMatches(
    img1, kp1,
    img2, kp2,
    good_matches,
    outImg=matches_img,
    singlePointColor=blue,
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_OVER_OUTIMG,
    matchesMask=inliers_mask,
    matchColor=green
)

# dibujo outliers
cv2.drawMatches(
    img1, kp1,
    img2, kp2,
    good_matches,
    outImg=matches_img,
    singlePointColor=blue,
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_OVER_OUTIMG,
    matchesMask=1 - inliers_mask,
    matchColor=red
)

imshow(matches_img, figsize=(12, 6))

## Approach cuadrados mínimos

- tenemos muchos más pares de correspondencias,
¿Cómo podríamos usarlos para encontrar una H mejor?


Dado que tenemos muchos más puntos, podríamos armar un sistema de ecuaciones mucho más grande incluyendo todos los pares matcheados.


Si tengo N matches, armaré un sistema lineal de 2 * N encuaciones (de manera análoga a DLT, pero con una sutil diferencia:

$ A . h =
\begin{bmatrix}
    -x_{(i)} & -y_{(i)} & -1 & 0 & 0 & 0 & x_{(i)} . x_{(i)}' & y_{(i)} . x_{(i)}' & x_{(i)}' \\
    0 & 0 & 0 & -x & -y & -1 & x . y' & y . y' & y_{(i)}' \\
\end{bmatrix}
.
\begin{bmatrix}
     h_{11}' \\
     h_{21}' \\
     h_{31}' \\
     h_{21}' \\
     h_{22}' \\
     h_{23}' \\
     h_{31}' \\
     h_{32}' \\
     h_{33}'
\end{bmatrix}
$

Resolveremos el sistema $ A . h = 0 $.
Como es homogéneo (el lado derecho es cero), podemos encontrar una solución trivial $h = 0$, pero esa solución no nos sirve. Por lo que se impone una restricción adicional.

Reformulamos el problema como:
minimizar $\|A h\|$ sujeto a $\|h\|_2 = 1$

Este es un patrón típico en este tipo de problemas, que se resuelve usando descomposición SVD de $A$:

$A = U \Sigma V^T$.

Se puede ver que la solución que minimiza $\|A h\|$ es el vector singular correspondiente al menor valor singular de A. Este vector es la última columna de $V$ (o la última fila de $V^T$)



In [ ]:
def estimar_homografia_svd(ori, dst):

    A = []
    b = []
    for i in range(len(ori)):
        x, y = ori[i]
        x_prima, y_prima = dst[i]
        A.append([-x, -y, -1, 0, 0, 0, x * x_prima, y * x_prima, x_prima])
        A.append([0, 0, 0, -x, -y, -1, x * y_prima, y * y_prima, y_prima])

    A = np.array(A)

    # calcula SVD
    _, _, Vt = np.linalg.svd(A)

    # La última fila de Vt (última columna de V) es la solución que minimiza ||A h||
    h = Vt[-1, :]

    H = h.reshape(3, 3)

    return H



In [ ]:
# probemos este enfoque:
use_matches = good_matches

In [ ]:
res_img = cv2.drawMatches(
    bw(img1), kp1,
    bw(img2), kp2,
    use_matches,
    None,
    matchColor=yellow,
    singlePointColor=red
)

imshow(res_img, figsize=(12, 6))

In [ ]:
pts1, pts2 = points_from_matches(kp1, kp2, use_matches)

# computo H usando SVD
H = estimar_homografia_svd(pts1, pts2)

In [ ]:
transformed_shape = transform(H, shape)

show_images([
    draw_shape(img1, shape),
    draw_shape(img2, transformed_shape)
])

In [ ]:
# uso todos los good-matches para calcular el error de reproyeccion
pts1, pts2 = points_from_matches(kp1, kp2, good_matches)

reprojection_error(H, pts1, pts2)

In [ ]:
# estamos mejor, pero podriamos mejorar más.
# cómo?

# RANSAC

RANdom SAmple Consensus (RANSAC), es un método iterativo para estimar parámetros de un modelo en un dataset que presenta gran cantidad de Outliers.

En cada iteración el algoritmo samplea de manera random la cantidad mínima de puntos necesaria para determinar el modelo, y luego testea en los datos cuántos puntos se ajustan bien al modelo. Los puntos que se ajustan bien son considerados inliers.


En este contexto, el modelo que se testeará en cada iteración será la Homografía que se calcula en cada paso usando DLT.


In [ ]:
des1 = des1.astype('float32')
des2 = des2.astype('float32')

In [ ]:
# OBS: pongo a propósito un macher "naive"
# de esta manera veremos la potencia del método de RANSAC.

# noisy matches

bf = cv2.BFMatcher(cv2.NORM_L2)
matches = bf.match(des1, des2)


# todos los matches.
draw_params = dict(
    matchColor = (255, 0, 0),
    singlePointColor = (0, 0, 255),  # draw unmatched keypoints in red color
)
res_img = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, **draw_params)

imshow(res_img, figsize=(12, 6))


In [ ]:
# probemos RANSAC (usando TODOS los matches!)
use_matches = matches
pts1, pts2 = points_from_matches(kp1, kp2, use_matches)

# encontramos la homografía usando RANSAC
H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, 5.0)

# Draw inliers matches (only matches where mask == 1)
matchesMask = mask.ravel().tolist()

# Draw inliers
draw_params = dict(matchColor=(0, 255, 0),  # draw matches in green color
                   singlePointColor=None,
                   matchesMask=matchesMask,  # draw only inliers
                   flags=2)

# Draw matches with inliers
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, **draw_params)

imshow(img_matches, figsize=(12, 6))

In [ ]:
# Y como da el monigote transformado?
transformed_shape = transform(H, shape)

show_images([
    draw_shape(img1, shape),
    draw_shape(img2, transformed_shape)
])

In [ ]:
# segun RANSAC cuales son los inliers?
inliers = []
for i, m in enumerate(matches):
    if matchesMask[i] == 1:
        inliers.append(m)

pts1, pts2 = points_from_matches(kp1, kp2, inliers)
reprojection_error(H, pts1, pts2)

# Ejercicio 2 - "realidad aumentada"
Queremos disimular nuestra pasión por la gastonomía y mostrar que estuvimos estudiando Visión Artificial.

Para eso, queremos reemplazar la portada del libro de Doña Petrona por la de Multiple View.

Contexto:
las dos primeras imagenes (las portadas de los libros) son conocidas, pero la tercera (la escena) es en runtime.
La solución debería poderse ejecutar en tiempo real con un stream de video de la cámara.



In [ ]:
img_petrona = cv2.imread("res/object_petrona.jpeg")
img_book = cv2.imread("res/multiple_view.jpg")
#img_escena = cv2.imread("res/scene13.jpeg")
img_escena = cv2.imread("res/scene1.jpeg")
img_result = cv2.imread("res/ejercicio2_result.jpg")

In [ ]:
show_images([
    img_petrona,
    img_book,
], ["doña petrona", "multiple view"])

In [ ]:
# El resultado esperado es:
show_images([
    img_escena,
    img_result,
], ["escena", "resultado esperado"])


In [ ]:
# Cómo hacemos?

# no vale mirar abajo sin antes pensar :)


In [ ]:
# TODO: completar lo que sigue:

# 0. podemos hacer lo de siempre buscar descriptores en el objeto y en la imagen...
#     suppngamos que tenemos los corners del libro en la imagen del objeto petrona,
#     entonces podríamos...
# 1. encontrar la homografía que mapea puntos de la tapa del libro petrona desde img_petrona hacia img_escena
# 2. transformar los corners del libro petrona hacia la escena
# 3. dibujar el contorno del libro petrona en la escena
# 4. warpear la imagen del libro petrona hacia la escena y graficarla
# 5. por otro lado encontrar una homografía que mapee el contorno del libro multiple-view a los corners de petrona.
# 6. encontrar la H compuesta que lleva del libro multiple view a petrona en la escena
# 7. warpear la imagen del libro multiple-view a la escena y graficar
# 8. usando los corners warpeados en la escena construir una máscara para aplicar la superposición.
#    para este paso se pueden usar las funciones de opencv fillConvexPoly, bitwise_not y bitwise_and
# 9. superponer la tapa de multiple view en la escena final.

In [ ]:
# supongamos que tenemos los corners del libro en la imagen del objeto petrona:

corners = [
    (210, 260),
    (1028, 230),
    (1028, 1433),
    (200, 1410),
]
corners = np.array(corners).reshape(1, -1, 2).astype('int32')

fig, ax = imshow(
    draw_shape(img_petrona, corners),
    figsize=(6, 8),
    show=False
)
pts = corners.reshape(-1, 2)
ax.scatter(
    pts[:, 0],
    pts[:, 1],
    color='r'
)
plt.show()


In [ ]:
algo = cv2.SIFT_create()

# find the keypoints and descriptors
kp1, des1 = algo.detectAndCompute(img_petrona, None)
kp2, des2 = algo.detectAndCompute(img_escena, None)

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
matches = bf.knnMatch(des1, des2, k=2)

# Nos quedamos con matches cuyo segundo mejor match está lejos
matchesMask = [[0, 0] for i in range(len(matches))]
good_matches = []
for i, (m, n) in enumerate(matches):
    if m.distance < 0.5 * n.distance:
        matchesMask[i] = [1, 0]
        good_matches.append(m)

# Dibuja sólo buenos matches
draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   matchesMask=matchesMask,
                   flags=cv2.DrawMatchesFlags_DEFAULT)


res_img = cv2.drawMatchesKnn(
    img_petrona, kp1, img_escena, kp2, matches, None, **draw_params
)

imshow(res_img, figsize=(20, 16))
plt.show()

In [ ]:
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ]).reshape(-1, 2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ]).reshape(-1, 2)

# 1. Find homography
H_petrona_escena, mask = cv2.findHomography(
    src_pts,
    dst_pts,
    cv2.RANSAC,
    5.0
)

In [ ]:
transformed_corners = transform(H_petrona_escena, corners)
transformed_corners

In [ ]:
dst_w, dst_h = img_escena.shape[1], img_escena.shape[0]
warped = cv2.warpPerspective(
    img_petrona,
    H_petrona_escena,
    (dst_w, dst_h)
)
show_escena = draw_shape(img_escena, transformed_corners)

In [ ]:
show_images([
    warped,
    show_escena
])

In [ ]:
h, w = img_book.shape[:2]
corners_book = np.array([[0, 0], [w, 0], [w, h], [0, h]], dtype=np.float32)

H_book_to_petrona = cv2.findHomography(corners_book, corners)[0]

H_book_to_escena = np.dot(H_petrona_escena, H_book_to_petrona)

# Warp img_book en img_escena
warped_book_cover = cv2.warpPerspective(
    img_book, H_book_to_escena,
    (dst_w, dst_h)
)

# Crea una máscara usano los corners warpeados para superponer en la escena
mask = np.zeros_like(img_escena, dtype=np.uint8)
cv2.fillConvexPoly(mask, transformed_corners, (255, 255, 255))

# En realidad necesitamos la máscara inversa
inverse_mask = cv2.bitwise_not(mask)

# Combina la imagen de la escena con la tapa de multiple view warpeada
scene_without_book = cv2.bitwise_and(img_escena, inverse_mask)
final_image = cv2.add(scene_without_book, warped_book_cover)

show_images([
    warped_book_cover,
    scene_without_book,
])

In [ ]:
show_images([
    img_escena,
    final_image
], ["original", "resultado"])